In [187]:
import numpy as np
import pandas as pd

In [188]:
data = pd.read_csv("./data/3_train.csv")
data.head(5)

,ID,area,ck,ck-saveall,ck-drawall,ck-savetime,ck-drawtime,ck-saveavg,ck-drawavg,ck-avg,...,dep-avg,dep-9201,fed-9201,fed-avg,comp,ck-changame,dep-changame,ck-changtime,dep-changtime,VV
0,1,104,?,0,0,0,0,?,NaN,0,...,0,NaN,0,0,.,0,0,.,0,Withheld
1,2,111,?,0,?,0,0,NaN,0,0,...,0,0,?,0,?,0,0,0,.,Withheld
2,8,600,0,?,0,NaN,0,?,0,NaN,...,.,?,NaN,0,0,0,0,?,?,Withheld
3,12,?,0,0,0,?,0,0,0,0,...,0,0,0,?,0,?,0,0,0,Withheld
4,13,?,0,0,0,0,0,0,0,0,...,130,0,?,0,0,0,0,0,0,Withheld


In [189]:
from Python.pandas.feature_selection import *
object_feature_helper(data)


---------------------------------------------
data contains [ 6537 ] records 
ID do not have any null values.
ID  is numeric column:
count     6537.000000
mean     16376.062720
std       9453.651871
min          1.000000
25%       8286.000000
50%      16359.000000
75%      24550.000000
max      32676.000000
Name: ID, dtype: float64



---------------------------------------------
area have 444 null values:

area  has [ 346 ] different values, please create features yourself!
please notice here are some seldom_indexes:  Index(['32700', '651', '437', '269', '735', '803', '963', '70300', '50600',
       '82200',
       ...
       '815', '913', '654', '635', '20300', '508', '82800', '260', '33700',
       '264'],
      dtype='object', length=102)



---------------------------------------------
ck have 425 null values:

0    3127
1    1731
?     826
.     428
Name: ck, dtype: int64
use following code to generate features from label mapping:
ck_mapping = {'0': 0, '1': 1, '?': 2, '.': 3}
#TO

In [190]:
def area_handler(area):
    if area in ['.', 'nan', '?']:
        return 0
    if len(area) < 3:
        return str(area) + "00"
    return str(area)[:3]

data_area = data['area'].apply(lambda area: area_handler(str(area)))
data_area1 = data_area.apply(lambda area: str(area)[:1])
from sklearn.preprocessing import OneHotEncoder
area_ohe = OneHotEncoder(drop='first')
data_area1_ohe = area_ohe.fit_transform(data_area1.values.reshape(-1,1))
data_area1_ohe = pd.DataFrame(data_area1_ohe.toarray(), index=data.index)
data_area1_ohe


,0,1,2,3,4,5,6,7,8
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6532,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
ck_mapping = {'0': 0, '1': 1, '?': 0, '.': 0}
data_ck, ck_ohe = one_hot_encoder_column(data, 'ck', ck_mapping, fill_na=0)

In [192]:
def numeric_to_int(x, na_val=-1):
    if x is None:
        return na_val
    if type(x) == str:
        if x in ['?', '.', 'NaN']:
            return na_val
        return int(x)
    if np.isnan(x):
        return na_val
    return x
def multi_fit(x, y, z):
    newx = numeric_to_int(x)
    newy = numeric_to_int(y)
    newz = numeric_to_int(z)
    if newx == -1:
        if newy != -1 and newz != -1:
            newx = newy * newz

    if newy == -1:
        if newx != -1 and newz > 0:
            newy = newx / newz
    if newz == -1:
        if newx != -1 and newy > 0:
            newz = newx / newy

    return newx, newy, newz
data_ck_saveall, data_ck_savetime, data_ck_saveavg = \
    zip(*data.apply(lambda row: multi_fit(row['ck-saveall'], row['ck-savetime'], row['ck-saveavg']), axis=1))
data_ckall = pd.concat([pd.Series(data_ck_saveall), pd.Series(data_ck_savetime), pd.Series(data_ck_saveavg)], axis=1)

In [193]:
data_dk_saveall, data_dk_savetime, data_dk_saveavg = \
    zip(*data.apply(lambda row: multi_fit(row['ck-drawall'], row['ck-drawtime'], row['ck-drawavg']), axis=1))
data_dkall = pd.concat([pd.Series(data_dk_saveall), pd.Series(data_dk_savetime), pd.Series(data_dk_saveavg)], axis=1)

In [194]:
data_ck_avg = data['ck-avg'].apply(lambda x: numeric_to_int(x, na_val=0))
data_ck_avg

0       0
1       0
2       0
3       0
4       0
       ..
6532    0
6533    0
6534    0
6535    0
6536    0
Name: ck-avg, Length: 6537, dtype: int64

In [195]:
data_dep_avg = data['dep-avg'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_avg

0           0
1           0
2           0
3           0
4         130
        ...  
6532      408
6533       18
6534    78933
6535        0
6536        0
Name: dep-avg, Length: 6537, dtype: int64

In [196]:
data_dep9201 = data['dep-9201'].apply(lambda x: numeric_to_int(x, na_val=0))
data_fed9201 = data['fed-9201'].apply(lambda x: numeric_to_int(x, na_val=0))
data_fed_avg = data['fed-avg'].apply(lambda x: numeric_to_int(x, na_val=0))

In [197]:
data_ck_changame = data['ck-changame'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_changame = data['dep-changame'].apply(lambda x: numeric_to_int(x, na_val=0))
data_ck_changtime = data['ck-changtime'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_changtime = data['dep-changtime'].apply(lambda x: numeric_to_int(x, na_val=0))

In [198]:
comp_mapping = {'0': 0, '?': 0, '1': 1, '.': 0}
data_comp, comp_ohe = one_hot_encoder_column(data, 'comp', comp_mapping, fill_na=0)

In [201]:
data_y = data['VV']

data_all = pd.concat([data_area1_ohe, data_ck, data_ckall, data_dkall, data_ck_avg, data_dep_avg,
                      data_dep9201, data_fed9201, data_fed_avg, data_ck_changame, data_dep_changame,
                      data_ck_changtime, data_dep_changtime, data_comp], axis=1)
data_all

,0,1,2,3,4,5,6,7,8,ck_0,...,ck-avg,dep-avg,dep-9201,fed-9201,fed-avg,ck-changame,dep-changame,ck-changtime,dep-changtime,comp_0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,130,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6532,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,408,0,0,0,0,0,0,0,0.0
6533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,18,0,0,0,0,0,0,0,0.0
6534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,78933,0,0,0,0,11250,0,2,0.0
6535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0


In [202]:
data_all.to_csv("./data/feature3_train.csv")